In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Project Predicting Flight Delays**

# Flights dataset

In [6]:
# import data
import pandas as pd

flights = pd.read_csv("/content/drive/MyDrive/HSG/project/flights.csv")
airlines = pd.read_csv("/content/drive/MyDrive/HSG/project/airlines.csv")
airports = pd.read_csv("/content/drive/MyDrive/HSG/project/airports.csv")
cancellation_codes = pd.read_csv("/content/drive/MyDrive/HSG/project/cancellation_codes.csv")

<ipython-input-6-24e39a442217>:4: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights = pd.read_csv("/content/drive/MyDrive/HSG/project/flights.csv")


In [7]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [8]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [10]:
airlines.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [11]:
cancellation_codes.head()

,CANCELLATION_REASON,CANCELLATION_DESCRIPTION
0,A,Airline/Carrier
1,B,Weather
2,C,National Air System
3,D,Security


In [12]:
# Count non-null weather delays for both origin and destination airports
origin_delays = flights.groupby("ORIGIN_AIRPORT")["WEATHER_DELAY"].count()
destination_delays = flights.groupby("DESTINATION_AIRPORT")["WEATHER_DELAY"].count()

# Combine the delays for origin and destination airports
combined_delays = origin_delays.add(destination_delays, fill_value=0)

# Get the top 10 airports by total weather delays (origin + destination)
top_airports = combined_delays.sort_values(ascending=False).head(10).index

# Filter the dataset for flights where both ORIGIN_AIRPORT and DESTINATION_AIRPORT are in the top 10
flights = flights[
    flights["ORIGIN_AIRPORT"].isin(top_airports) &
    flights["DESTINATION_AIRPORT"].isin(top_airports)
]

# Display the top airports data
flights.info()

<class 'pandas.core.frame.DataFrame'>
Index: 460390 entries, 8 to 5819068
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   YEAR                 460390 non-null  int64  
 1   MONTH                460390 non-null  int64  
 2   DAY                  460390 non-null  int64  
 3   DAY_OF_WEEK          460390 non-null  int64  
 4   AIRLINE              460390 non-null  object 
 5   FLIGHT_NUMBER        460390 non-null  int64  
 6   TAIL_NUMBER          458321 non-null  object 
 7   ORIGIN_AIRPORT       460390 non-null  object 
 8   DESTINATION_AIRPORT  460390 non-null  object 
 9   SCHEDULED_DEPARTURE  460390 non-null  int64  
 10  DEPARTURE_TIME       454013 non-null  float64
 11  DEPARTURE_DELAY      454013 non-null  float64
 12  TAXI_OUT             453750 non-null  float64
 13  WHEELS_OFF           453750 non-null  float64
 14  SCHEDULED_TIME       460389 non-null  float64
 15  ELAPSED_TIME         

In [13]:
#date column
flights["date"] = pd.to_datetime(flights[["YEAR", "MONTH", "DAY"]])

#merge airline names
flights = pd.merge(flights, airlines, left_on="AIRLINE", right_on="IATA_CODE", how="left")

#merge airports ORIGIN
flights = pd.merge(flights, airports, left_on="ORIGIN_AIRPORT", right_on="IATA_CODE", how="left")
flights.rename(columns=lambda col: f"Origin_{col}" if col in airports.columns else col, inplace=True)

#merge airports DESTINATION
flights = pd.merge(flights, airports, left_on="DESTINATION_AIRPORT", right_on="IATA_CODE", how="left")
flights.rename(columns=lambda col: f"Destination_{col}" if col in airports.columns else col, inplace=True)

#merge cancellation codes
flights = pd.merge(flights, cancellation_codes, on="CANCELLATION_REASON", how="left")


flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,Origin_LATITUDE,Origin_LONGITUDE,Destination_IATA_CODE,Destination_AIRPORT,Destination_CITY,Destination_STATE,Destination_COUNTRY,Destination_LATITUDE,Destination_LONGITUDE,CANCELLATION_DESCRIPTION
0,2015,1,1,4,AA,1112,N3LAAA,SFO,DFW,30,...,37.61900,-122.37484,DFW,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720,NaN
1,2015,1,1,4,DL,1173,N826DN,LAS,ATL,30,...,36.08036,-115.15233,ATL,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,NaN
2,2015,1,1,4,DL,2336,N958DN,DEN,ATL,30,...,39.85841,-104.66700,ATL,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,NaN
3,2015,1,1,4,UA,1197,N78448,SFO,IAH,48,...,37.61900,-122.37484,IAH,George Bush Intercontinental Airport,Houston,TX,USA,29.98047,-95.33972,NaN
4,2015,1,1,4,NK,214,N632NK,LAS,DFW,103,...,36.08036,-115.15233,DFW,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720,NaN


In [14]:
print(flights.columns)


Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE_x', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'date', 'IATA_CODE_x',
       'AIRLINE_y', 'IATA_CODE_y', 'Origin_AIRPORT', 'Origin_CITY',
       'Origin_STATE', 'Origin_COUNTRY', 'Origin_LATITUDE', 'Origin_LONGITUDE',
       'Destination_IATA_CODE', 'Destination_AIRPORT', 'Destination_CITY',
       'Destination_STATE', 'Destination_COUNTRY', 'Destination_LATITUDE',
       'Destination_LONGITUDE', 'CANCELLATION_DESCRIPTION'],
      dtype='object')


In [15]:
print(top_airports)

Index(['ORD', 'ATL', 'DFW', 'LAX', 'DEN', 'SFO', 'IAH', 'PHX', 'LAS', 'LGA'], dtype='object')


In [16]:
top10_airports = airports[airports["IATA_CODE"].isin(top_airports)]
top10_airports

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
20,ATL,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694
85,DEN,Denver International Airport,Denver,CO,USA,39.85841,-104.66700
86,DFW,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720
151,IAH,George Bush Intercontinental Airport,Houston,TX,USA,29.98047,-95.33972
174,LAS,McCarran International Airport,Las Vegas,NV,USA,36.08036,-115.15233
176,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
182,LGA,LaGuardia Airport (Marine Air Terminal),New York,NY,USA,40.77724,-73.87261
228,ORD,Chicago O'Hare International Airport,Chicago,IL,USA,41.97960,-87.90446
239,PHX,Phoenix Sky Harbor International Airport,Phoenix,AZ,USA,33.43417,-112.00806
278,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484


In [17]:
# Data for the airports
data = {
    "IATA_CODE": ["ATL", "DEN", "DFW", "IAH", "LAS", "LAX", "LGA", "ORD", "PHX", "SFO"],
    "UTC_OFFSET": [-5, -7, -6, -6, -8, -8, -5, -6, -7, -8]  # Approximate UTC offsets
}

# Create a DataFrame
airport_time_diff = pd.DataFrame(data)

# View the DataFrame
airport_time_diff

,IATA_CODE,UTC_OFFSET
0,ATL,-5
1,DEN,-7
2,DFW,-6
3,IAH,-6
4,LAS,-8
5,LAX,-8
6,LGA,-5
7,ORD,-6
8,PHX,-7
9,SFO,-8


In [18]:
# Create a matrix of time differences
time_diff_matrix = pd.DataFrame(
    [
        [
            airport_time_diff.loc[i, "UTC_OFFSET"] - airport_time_diff.loc[j, "UTC_OFFSET"]
            for j in range(len(airport_time_diff))
        ]
        for i in range(len(airport_time_diff))
    ],
    index=airport_time_diff["IATA_CODE"],
    columns=airport_time_diff["IATA_CODE"]
)

# View the matrix
time_diff_matrix

IATA_CODE,ATL,DEN,DFW,IAH,LAS,LAX,LGA,ORD,PHX,SFO
IATA_CODE,,,,,,,,,,
ATL,0,2,1,1,3,3,0,1,2,3
DEN,-2,0,-1,-1,1,1,-2,-1,0,1
DFW,-1,1,0,0,2,2,-1,0,1,2
IAH,-1,1,0,0,2,2,-1,0,1,2
LAS,-3,-1,-2,-2,0,0,-3,-2,-1,0
LAX,-3,-1,-2,-2,0,0,-3,-2,-1,0
LGA,0,2,1,1,3,3,0,1,2,3
ORD,-1,1,0,0,2,2,-1,0,1,2
PHX,-2,0,-1,-1,1,1,-2,-1,0,1


In [19]:
# Convert the matrix to a two-column format
time_diff_long = time_diff_matrix.reset_index().melt(
    id_vars="IATA_CODE",
    var_name="Destination",
    value_name="Time Difference"
)

# Rename the origin column
time_diff_long.rename(columns={"IATA_CODE": "Origin"}, inplace=True)

# Multiply the 'Time Difference' values by -1
time_diff_long['Time Difference'] = time_diff_long['Time Difference'] * -1

# View the transformed DataFrame
time_diff_long


,Origin,Destination,Time Difference
0,ATL,ATL,0
1,DEN,ATL,2
2,DFW,ATL,1
3,IAH,ATL,1
4,LAS,ATL,3
...,...,...,...
95,LAX,SFO,0
96,LGA,SFO,-3
97,ORD,SFO,-2
98,PHX,SFO,-1


In [20]:
# Convert SCHEDULED_TIME and ELAPSED_TIME to hours and create new columns
flights["SCHEDULED_TIME_HOURS"] = flights["SCHEDULED_TIME"] / 60
flights["ELAPSED_TIME_HOURS"] = flights["ELAPSED_TIME"] / 60

# Verify the changes
flights[["SCHEDULED_TIME", "ELAPSED_TIME", "SCHEDULED_TIME_HOURS", "ELAPSED_TIME_HOURS"]].head()

,SCHEDULED_TIME,ELAPSED_TIME,SCHEDULED_TIME_HOURS,ELAPSED_TIME_HOURS
0,195.0,193.0,3.250000,3.216667
1,221.0,203.0,3.683333,3.383333
2,173.0,149.0,2.883333,2.483333
3,218.0,217.0,3.633333,3.616667
4,147.0,147.0,2.450000,2.450000


In [21]:
# Merge the time difference data into the flights DataFrame
flights = flights.merge(
    time_diff_long,
    left_on=["ORIGIN_AIRPORT", "DESTINATION_AIRPORT"],
    right_on=["Origin", "Destination"],
    how="left"
)

# Drop the redundant columns from the merge
flights.drop(columns=["Origin", "Destination"], inplace=True)

# View the updated flights DataFrame
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,Destination_AIRPORT,Destination_CITY,Destination_STATE,Destination_COUNTRY,Destination_LATITUDE,Destination_LONGITUDE,CANCELLATION_DESCRIPTION,SCHEDULED_TIME_HOURS,ELAPSED_TIME_HOURS,Time Difference
0,2015,1,1,4,AA,1112,N3LAAA,SFO,DFW,30,...,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720,NaN,3.250000,3.216667,2
1,2015,1,1,4,DL,1173,N826DN,LAS,ATL,30,...,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,NaN,3.683333,3.383333,3
2,2015,1,1,4,DL,2336,N958DN,DEN,ATL,30,...,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,NaN,2.883333,2.483333,2
3,2015,1,1,4,UA,1197,N78448,SFO,IAH,48,...,George Bush Intercontinental Airport,Houston,TX,USA,29.98047,-95.33972,NaN,3.633333,3.616667,2
4,2015,1,1,4,NK,214,N632NK,LAS,DFW,103,...,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720,NaN,2.450000,2.450000,2


In [22]:
flights['SCHEDULED_TIME_BOOL'] = (
    (flights['Time Difference'] < 0) &
    (-flights['SCHEDULED_TIME_HOURS'] > flights['Time Difference'])
).astype(int)  # Convert boolean to integer (0 or 1)

flights['ELAPSED_TIME_BOOL'] = (
    (flights['Time Difference'] < 0) &
    (-flights['ELAPSED_TIME_HOURS'] > flights['Time Difference'])
).astype(int)  # Convert boolean to integer (0 or 1)

# View the updated DataFrame
flights[['SCHEDULED_TIME_HOURS', 'ELAPSED_TIME_HOURS', 'Time Difference', 'SCHEDULED_TIME_BOOL', 'ELAPSED_TIME_BOOL']]

,SCHEDULED_TIME_HOURS,ELAPSED_TIME_HOURS,Time Difference,SCHEDULED_TIME_BOOL,ELAPSED_TIME_BOOL
0,3.250000,3.216667,2,0,0
1,3.683333,3.383333,3,0,0
2,2.883333,2.483333,2,0,0
3,3.633333,3.616667,2,0,0
4,2.450000,2.450000,2,0,0
...,...,...,...,...,...
460385,3.700000,3.366667,2,0,0
460386,4.066667,3.816667,3,0,0
460387,2.916667,2.450000,2,0,0
460388,4.000000,3.666667,2,0,0


In [23]:
bool_filter = flights[(flights['SCHEDULED_TIME_BOOL'] == 1) | (flights['ELAPSED_TIME_BOOL'] == 1)]

# Display the filtered DataFrame
bool_filter

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,Destination_STATE,Destination_COUNTRY,Destination_LATITUDE,Destination_LONGITUDE,CANCELLATION_DESCRIPTION,SCHEDULED_TIME_HOURS,ELAPSED_TIME_HOURS,Time Difference,SCHEDULED_TIME_BOOL,ELAPSED_TIME_BOOL
634,2015,1,1,4,WN,658,N8621A,PHX,LAS,1405,...,NV,USA,36.08036,-115.15233,NaN,1.083333,0.983333,-1,0,1
890,2015,1,1,4,WN,738,N251WN,PHX,LAS,1715,...,NV,USA,36.08036,-115.15233,NaN,1.083333,0.983333,-1,0,1
1118,2015,1,1,4,WN,636,N650SW,PHX,LAS,2025,...,NV,USA,36.08036,-115.15233,NaN,1.000000,0.900000,-1,0,1
1602,2015,1,2,5,WN,1517,N271LV,PHX,LAS,935,...,NV,USA,36.08036,-115.15233,NaN,1.083333,0.883333,-1,0,1
1785,2015,1,2,5,WN,3450,N434WN,PHX,LAS,1140,...,NV,USA,36.08036,-115.15233,NaN,1.083333,0.900000,-1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459156,2015,12,30,3,WN,2327,N945WN,PHX,LAS,2230,...,NV,USA,36.08036,-115.15233,NaN,1.000000,0.883333,-1,0,1
459181,2015,12,30,3,AA,1881,N582UW,PHX,LAS,2345,...,NV,USA,36.08036,-115.15233,NaN,1.116667,0.983333,-1,0,1
459995,2015,12,31,4,WN,5126,N646SW,PHX,LAS,1515,...,NV,USA,36.08036,-115.15233,NaN,1.083333,0.933333,-1,0,1
460103,2015,12,31,4,WN,5197,N643SW,PHX,LAS,1650,...,NV,USA,36.08036,-115.15233,NaN,1.083333,0.883333,-1,0,1


In [24]:
import pandas as pd
import numpy as np

# Function to convert time in the format like 2350 -> 23:50, 25 -> 00:25, etc.
def convert_to_time_format(time_value):
    if pd.isna(time_value):
        return np.nan  # Return NaN if the value is NaN
    time_value = int(time_value)  # Convert to integer if it's a string
    hours = time_value // 100
    minutes = time_value % 100

    # Ensure the minutes part is less than 60
    if minutes >= 60:
        hours += minutes // 60
        minutes = minutes % 60

    # Ensure the hours and minutes are in two-digit format
    return f"{int(hours):02d}:{int(minutes):02d}"

# Apply the function to convert the time columns in the flights dataframe
time_columns = ['SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'ARRIVAL_TIME', 'SCHEDULED_ARRIVAL']
for col in time_columns:
    flights[col] = flights[col].apply(convert_to_time_format)

# Function to combine the 'date' column and time columns to get datetime
def combine_date_and_time(date, time_str):
    if pd.isna(time_str):
        return np.nan
    try:
        # Convert time to datetime format
        time = pd.to_datetime(time_str, format='%H:%M').time()
        # Combine date with time
        combined_datetime = pd.to_datetime(date) + pd.Timedelta(hours=time.hour, minutes=time.minute)
        return combined_datetime
    except Exception as e:
        return np.nan  # Return NaN if there's an error in parsing

# Apply the function to combine the 'date' column with the time columns
for col in time_columns:
    flights[f'{col}_DATETIME'] = flights.apply(
        lambda row: combine_date_and_time(row['date'], row[col]), axis=1
    )


In [25]:
# Convert '24:00' to '00:00' in all relevant columns
flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].replace('24:00', '00:00')
flights['ARRIVAL_TIME'] = flights['ARRIVAL_TIME'].replace('24:00', '00:00')
flights['SCHEDULED_DEPARTURE'] = flights['SCHEDULED_DEPARTURE'].replace('24:00', '00:00')
flights['SCHEDULED_ARRIVAL'] = flights['SCHEDULED_ARRIVAL'].replace('24:00', '00:00')

# Convert the time columns to datetime
flights['DEPARTURE_TIME_DATETIME'] = pd.to_datetime(flights['date'].astype(str) + ' ' + flights['DEPARTURE_TIME'], errors='coerce')
flights['ARRIVAL_TIME_DATETIME'] = pd.to_datetime(flights['date'].astype(str) + ' ' + flights['ARRIVAL_TIME'], errors='coerce')
flights['SCHEDULED_DEPARTURE_DATETIME'] = pd.to_datetime(flights['date'].astype(str) + ' ' + flights['SCHEDULED_DEPARTURE'], errors='coerce')
flights['SCHEDULED_ARRIVAL_DATETIME'] = pd.to_datetime(flights['date'].astype(str) + ' ' + flights['SCHEDULED_ARRIVAL'], errors='coerce')

# Ensure DEPARTURE_TIME_DATETIME is always smaller than ARRIVAL_TIME_DATETIME
# Apply the adjustment only when the associated boolean column is 0 (False)
flights['ARRIVAL_TIME_DATETIME'] = np.where(
    (flights['DEPARTURE_TIME_DATETIME'] > flights['ARRIVAL_TIME_DATETIME']) & (flights['ELAPSED_TIME_BOOL'] == 0),
    flights['ARRIVAL_TIME_DATETIME'] + pd.Timedelta(days=1),
    flights['ARRIVAL_TIME_DATETIME']
)

# Ensure SCHEDULED_DEPARTURE_DATETIME is always smaller than SCHEDULED_ARRIVAL_DATETIME
# Apply the adjustment only when the associated boolean column is 0 (False)
flights['SCHEDULED_ARRIVAL_DATETIME'] = np.where(
    (flights['SCHEDULED_DEPARTURE_DATETIME'] > flights['SCHEDULED_ARRIVAL_DATETIME']) & (flights['SCHEDULED_TIME_BOOL'] == 0),
    flights['SCHEDULED_ARRIVAL_DATETIME'] + pd.Timedelta(days=1),
    flights['SCHEDULED_ARRIVAL_DATETIME']
)

# Display the updated columns to check if the changes were applied correctly
flights[['DEPARTURE_TIME', 'DEPARTURE_TIME_DATETIME', 'ARRIVAL_TIME', 'ARRIVAL_TIME_DATETIME',
         'SCHEDULED_DEPARTURE', 'SCHEDULED_DEPARTURE_DATETIME', 'SCHEDULED_ARRIVAL', 'SCHEDULED_ARRIVAL_DATETIME', "SCHEDULED_TIME_HOURS",	"ELAPSED_TIME_HOURS", "Time Difference", "SCHEDULED_TIME_BOOL",	"ELAPSED_TIME_BOOL"]]

,DEPARTURE_TIME,DEPARTURE_TIME_DATETIME,ARRIVAL_TIME,ARRIVAL_TIME_DATETIME,SCHEDULED_DEPARTURE,SCHEDULED_DEPARTURE_DATETIME,SCHEDULED_ARRIVAL,SCHEDULED_ARRIVAL_DATETIME,SCHEDULED_TIME_HOURS,ELAPSED_TIME_HOURS,Time Difference,SCHEDULED_TIME_BOOL,ELAPSED_TIME_BOOL
0,00:19,2015-01-01 00:19:00,05:32,2015-01-01 05:32:00,00:30,2015-01-01 00:30:00,05:45,2015-01-01 05:45:00,3.250000,3.216667,2,0,0
1,00:33,2015-01-01 00:33:00,06:56,2015-01-01 06:56:00,00:30,2015-01-01 00:30:00,07:11,2015-01-01 07:11:00,3.683333,3.383333,3,0,0
2,00:24,2015-01-01 00:24:00,04:53,2015-01-01 04:53:00,00:30,2015-01-01 00:30:00,05:23,2015-01-01 05:23:00,2.883333,2.483333,2,0,0
3,00:42,2015-01-01 00:42:00,06:19,2015-01-01 06:19:00,00:48,2015-01-01 00:48:00,06:26,2015-01-01 06:26:00,3.633333,3.616667,2,0,0
4,01:02,2015-01-01 01:02:00,05:29,2015-01-01 05:29:00,01:03,2015-01-01 01:03:00,05:30,2015-01-01 05:30:00,2.450000,2.450000,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
460385,23:37,2015-12-31 23:37:00,04:59,2016-01-01 04:59:00,23:43,2015-12-31 23:43:00,05:25,2016-01-01 05:25:00,3.700000,3.366667,2,0,0
460386,00:16,2015-12-31 00:16:00,07:05,2015-12-31 07:05:00,23:50,2015-12-31 23:50:00,06:54,2016-01-01 06:54:00,4.066667,3.816667,3,0,0
460387,00:14,2015-12-31 00:14:00,04:41,2015-12-31 04:41:00,23:55,2015-12-31 23:55:00,04:50,2016-01-01 04:50:00,2.916667,2.450000,2,0,0
460388,00:01,2015-12-31 00:01:00,05:41,2015-12-31 05:41:00,23:55,2015-12-31 23:55:00,05:55,2016-01-01 05:55:00,4.000000,3.666667,2,0,0


In [26]:
# Filter rows where either 'SCHEDULED_TIME_BOOL' or 'ELAPSED_TIME_BOOL' is 1
filtered_flights = flights[(flights['SCHEDULED_TIME_BOOL'] == 1) | (flights['ELAPSED_TIME_BOOL'] == 1)]

# Display the specific columns from the filtered dataframe
filtered_columns = [
    'DEPARTURE_TIME', 'DEPARTURE_TIME_DATETIME', 'ARRIVAL_TIME', 'ARRIVAL_TIME_DATETIME',
    'SCHEDULED_DEPARTURE', 'SCHEDULED_DEPARTURE_DATETIME', 'SCHEDULED_ARRIVAL', 'SCHEDULED_ARRIVAL_DATETIME',
    "SCHEDULED_TIME_HOURS", "ELAPSED_TIME_HOURS", "Time Difference", "SCHEDULED_TIME_BOOL", "ELAPSED_TIME_BOOL", "ORIGIN_AIRPORT",	"DESTINATION_AIRPORT"
]

# Display the filtered dataframe with the selected columns
filtered_flights_display = filtered_flights[filtered_columns]
display(filtered_flights_display)

,DEPARTURE_TIME,DEPARTURE_TIME_DATETIME,ARRIVAL_TIME,ARRIVAL_TIME_DATETIME,SCHEDULED_DEPARTURE,SCHEDULED_DEPARTURE_DATETIME,SCHEDULED_ARRIVAL,SCHEDULED_ARRIVAL_DATETIME,SCHEDULED_TIME_HOURS,ELAPSED_TIME_HOURS,Time Difference,SCHEDULED_TIME_BOOL,ELAPSED_TIME_BOOL,ORIGIN_AIRPORT,DESTINATION_AIRPORT
634,14:17,2015-01-01 14:17:00,14:16,2015-01-01 14:16:00,14:05,2015-01-01 14:05:00,14:10,2015-01-01 14:10:00,1.083333,0.983333,-1,0,1,PHX,LAS
890,17:15,2015-01-01 17:15:00,17:14,2015-01-01 17:14:00,17:15,2015-01-01 17:15:00,17:20,2015-01-01 17:20:00,1.083333,0.983333,-1,0,1,PHX,LAS
1118,20:25,2015-01-01 20:25:00,20:19,2015-01-01 20:19:00,20:25,2015-01-01 20:25:00,20:25,2015-01-01 20:25:00,1.000000,0.900000,-1,0,1,PHX,LAS
1602,09:47,2015-01-02 09:47:00,09:40,2015-01-02 09:40:00,09:35,2015-01-02 09:35:00,09:40,2015-01-02 09:40:00,1.083333,0.883333,-1,0,1,PHX,LAS
1785,12:09,2015-01-02 12:09:00,12:03,2015-01-02 12:03:00,11:40,2015-01-02 11:40:00,11:45,2015-01-02 11:45:00,1.083333,0.900000,-1,0,1,PHX,LAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459156,01:09,2015-12-30 01:09:00,01:02,2015-12-30 01:02:00,22:30,2015-12-30 22:30:00,22:30,2015-12-30 22:30:00,1.000000,0.883333,-1,0,1,PHX,LAS
459181,23:45,2015-12-30 23:45:00,23:44,2015-12-30 23:44:00,23:45,2015-12-30 23:45:00,23:52,2015-12-30 23:52:00,1.116667,0.983333,-1,0,1,PHX,LAS
459995,15:38,2015-12-31 15:38:00,15:34,2015-12-31 15:34:00,15:15,2015-12-31 15:15:00,15:20,2015-12-31 15:20:00,1.083333,0.933333,-1,0,1,PHX,LAS
460103,16:58,2015-12-31 16:58:00,16:51,2015-12-31 16:51:00,16:50,2015-12-31 16:50:00,16:55,2015-12-31 16:55:00,1.083333,0.883333,-1,0,1,PHX,LAS


In [27]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460390 entries, 0 to 460389
Data columns (total 58 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   YEAR                          460390 non-null  int64         
 1   MONTH                         460390 non-null  int64         
 2   DAY                           460390 non-null  int64         
 3   DAY_OF_WEEK                   460390 non-null  int64         
 4   AIRLINE_x                     460390 non-null  object        
 5   FLIGHT_NUMBER                 460390 non-null  int64         
 6   TAIL_NUMBER                   458321 non-null  object        
 7   ORIGIN_AIRPORT                460390 non-null  object        
 8   DESTINATION_AIRPORT           460390 non-null  object        
 9   SCHEDULED_DEPARTURE           460390 non-null  object        
 10  DEPARTURE_TIME                454013 non-null  object        
 11  DEPARTURE_DEL

In [28]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,CANCELLATION_DESCRIPTION,SCHEDULED_TIME_HOURS,ELAPSED_TIME_HOURS,Time Difference,SCHEDULED_TIME_BOOL,ELAPSED_TIME_BOOL,SCHEDULED_DEPARTURE_DATETIME,DEPARTURE_TIME_DATETIME,ARRIVAL_TIME_DATETIME,SCHEDULED_ARRIVAL_DATETIME
0,2015,1,1,4,AA,1112,N3LAAA,SFO,DFW,00:30,...,NaN,3.250000,3.216667,2,0,0,2015-01-01 00:30:00,2015-01-01 00:19:00,2015-01-01 05:32:00,2015-01-01 05:45:00
1,2015,1,1,4,DL,1173,N826DN,LAS,ATL,00:30,...,NaN,3.683333,3.383333,3,0,0,2015-01-01 00:30:00,2015-01-01 00:33:00,2015-01-01 06:56:00,2015-01-01 07:11:00
2,2015,1,1,4,DL,2336,N958DN,DEN,ATL,00:30,...,NaN,2.883333,2.483333,2,0,0,2015-01-01 00:30:00,2015-01-01 00:24:00,2015-01-01 04:53:00,2015-01-01 05:23:00
3,2015,1,1,4,UA,1197,N78448,SFO,IAH,00:48,...,NaN,3.633333,3.616667,2,0,0,2015-01-01 00:48:00,2015-01-01 00:42:00,2015-01-01 06:19:00,2015-01-01 06:26:00
4,2015,1,1,4,NK,214,N632NK,LAS,DFW,01:03,...,NaN,2.450000,2.450000,2,0,0,2015-01-01 01:03:00,2015-01-01 01:02:00,2015-01-01 05:29:00,2015-01-01 05:30:00


In [29]:
flights.to_csv('/content/drive/MyDrive/HSG/project/flights_final.csv', index=False)

# Weather dataset

In [30]:
#import weather data
ATL = pd.read_csv("/content/drive/MyDrive/HSG/project/ATL.csv")
CAL = pd.read_csv("/content/drive/MyDrive/HSG/project/CALIFORNIA.csv")
DEN = pd.read_csv("/content/drive/MyDrive/HSG/project/DEN.csv")
LAS = pd.read_csv("/content/drive/MyDrive/HSG/project/LAS.csv")
LGA = pd.read_csv("/content/drive/MyDrive/HSG/project/LGA.csv")
ORD = pd.read_csv("/content/drive/MyDrive/HSG/project/ORD.csv")
PHX = pd.read_csv("/content/drive/MyDrive/HSG/project/PHX.csv")
TEX = pd.read_csv("/content/drive/MyDrive/HSG/project/TEXAS.csv")

In [31]:
#stack the dataframes
weather = pd.concat([ATL, CAL, DEN, LAS, LGA, ORD, PHX, TEX], ignore_index=True)

In [32]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   station  99224 non-null  object
 1   valid    99224 non-null  object
 2   tmpc     99224 non-null  object
 3   sped     99224 non-null  object
 4   p01m     99224 non-null  object
 5   vsby     99224 non-null  object
dtypes: object(6)
memory usage: 4.5+ MB


In [33]:
#adjust data format
weather['valid'] = pd.to_datetime(weather['valid'])
weather['tmpc'] = pd.to_numeric(weather['tmpc'], errors='coerce')  # Handling invalid entries by coercing them to NaN
weather['sped'] = pd.to_numeric(weather['sped'], errors='coerce')
weather['vsby'] = pd.to_numeric(weather['vsby'], errors='coerce')
weather['p01m'] = pd.to_numeric(weather['p01m'], errors='coerce')


In [34]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   station  99224 non-null  object        
 1   valid    99224 non-null  datetime64[ns]
 2   tmpc     99212 non-null  float64       
 3   sped     99104 non-null  float64       
 4   p01m     99223 non-null  float64       
 5   vsby     99210 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 4.5+ MB


In [35]:
null_counts = weather.isnull().sum()
print(null_counts)

station      0
valid        0
tmpc        12
sped       120
p01m         1
vsby        14
dtype: int64


In [36]:
flights_with_missing_values = weather[weather.isnull().any(axis=1)]
flights_with_missing_values

,station,valid,tmpc,sped,p01m,vsby
1066,ATL,2015-02-06 23:52:00,4.44,NaN,0.0000,10.0
8581,ATL,2015-10-17 13:52:00,20.00,NaN,0.0000,10.0
9195,ATL,2015-11-07 03:32:00,21.00,NaN,0.0000,8.0
9368,ATL,2015-11-12 08:18:00,17.22,NaN,0.0001,2.0
9604,ATL,2015-11-20 11:52:00,18.33,NaN,0.0000,10.0
...,...,...,...,...,...,...
92227,DFW,2015-08-27 12:53:00,33.33,NaN,0.0000,10.0
92782,DFW,2015-09-07 12:53:00,36.11,NaN,0.0000,10.0
93760,DFW,2015-09-26 13:53:00,31.11,NaN,0.0000,10.0
94241,DFW,2015-10-06 14:53:00,27.22,NaN,0.0000,10.0


In [37]:
# Count occurrences of each station
station_counts = flights_with_missing_values['station'].value_counts()

# View the result
station_counts


,count
station,
PHX,54
LAS,28
DFW,15
DEN,12
ATL,5
ORD,4
LAX,3
LGA,3
SFO,1


In [38]:
# Create a mask where True indicates missing values
missing_mask = weather.isnull().any(axis=1)

# Check for consecutive missing values (back-to-back rows with missing values)
back_to_back_missing = missing_mask & missing_mask.shift(-1, fill_value=False)

# Get the rows with consecutive missing values
back_to_back_missing_data = weather[back_to_back_missing]

# Display the first few rows of back-to-back missing values
print(back_to_back_missing_data)

      station               valid   tmpc  sped  p01m  vsby
35096     DEN 2024-06-21 11:53:00  23.89   NaN   0.0  10.0
75354     PHX 2015-08-31 22:51:00    NaN   NaN   0.0   NaN
75355     PHX 2015-08-31 23:51:00    NaN   NaN   0.0   NaN
75356     PHX 2015-09-01 00:16:00    NaN   NaN   0.0   NaN
75357     PHX 2015-09-01 00:51:00    NaN   NaN   0.0   NaN
75358     PHX 2015-09-01 01:51:00    NaN   NaN   0.0   NaN
75359     PHX 2015-09-01 02:51:00    NaN   NaN   0.0   NaN
75362     PHX 2015-09-01 04:51:00    NaN   NaN   0.0   NaN


In [39]:
# Filter rows where the 'valid' date is either January 1st or December 31st
jan_1_dec_31_entries = flights_with_missing_values[
    (flights_with_missing_values['valid'].dt.month == 1) & (flights_with_missing_values['valid'].dt.day == 1) |
    (flights_with_missing_values['valid'].dt.month == 12) & (flights_with_missing_values['valid'].dt.day == 31)
]

# Display the first few rows of the filtered data
print(jan_1_dec_31_entries)

Empty DataFrame
Columns: [station, valid, tmpc, sped, p01m, vsby]
Index: []


In [40]:
# Define a function to fill NaN values with the average of the nearest non-null values above and below
def fill_na_with_average_of_nearest(df):
    # Only apply filling to numeric columns, skip datetime columns like 'valid'
    for col in df.select_dtypes(include=['float64']):
        # Forward fill to get the value above the NaN
        forward_filled = df[col].ffill()

        # Backward fill to get the value below the NaN
        backward_filled = df[col].bfill()

        # Now fill NaN by taking the average of forward and backward filled values
        df[col] = (forward_filled + backward_filled) / 2

    return df

# Apply the function to the dataframe
weather = fill_na_with_average_of_nearest(weather)

# Display the updated dataframe
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   station  99224 non-null  object        
 1   valid    99224 non-null  datetime64[ns]
 2   tmpc     99224 non-null  float64       
 3   sped     99224 non-null  float64       
 4   p01m     99224 non-null  float64       
 5   vsby     99224 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 4.5+ MB


In [41]:
weather.to_csv('/content/drive/MyDrive/HSG/project/weather_final.csv', index=False)

# Merge both datasets

In [1]:
import pandas as pd
import numpy as np

In [2]:
flights = pd.read_csv("/content/drive/MyDrive/HSG/project/flights_final.csv")
weather = pd.read_csv("/content/drive/MyDrive/HSG/project/weather_final.csv")

In [3]:
# Ensure the datetime columns in both DataFrames are in proper datetime format
flights['SCHEDULED_DEPARTURE_DATETIME'] = pd.to_datetime(flights['SCHEDULED_DEPARTURE_DATETIME'], errors='coerce')
flights['SCHEDULED_ARRIVAL_DATETIME'] = pd.to_datetime(flights['SCHEDULED_ARRIVAL_DATETIME'], errors='coerce')
flights['DEPARTURE_TIME_DATETIME'] = pd.to_datetime(flights['DEPARTURE_TIME_DATETIME'], errors='coerce')
flights['ARRIVAL_TIME_DATETIME'] = pd.to_datetime(flights['ARRIVAL_TIME_DATETIME'], errors='coerce')
weather['valid'] = pd.to_datetime(weather['valid'], errors='coerce')

In [4]:
from datetime import timedelta

# Define the time shifts
time_shifts = [-6, -1, 1, 6]

# Function to create shifted datetime columns
def create_shifted_datetime(df, base_column, shifts, prefix):
    for shift in shifts:
        shift_label = f"{prefix}_{abs(shift)}hr_{'before' if shift < 0 else 'after'}"
        df[shift_label] = df[base_column] + timedelta(hours=shift)
    return df

# Create shifted datetime columns for departure and arrival
flights = create_shifted_datetime(flights, 'SCHEDULED_DEPARTURE_DATETIME', time_shifts, 'departure')
flights = create_shifted_datetime(flights, 'SCHEDULED_ARRIVAL_DATETIME', time_shifts, 'arrival')

In [31]:
# List of all legged departure and arrival columns (including the newly added time shifts)
departure_columns = [
    'SCHEDULED_DEPARTURE_DATETIME', 'departure_6hr_before', 'departure_1hr_before',
    'departure_1hr_after', 'departure_6hr_after'
]

arrival_columns = [
    'SCHEDULED_ARRIVAL_DATETIME', 'arrival_6hr_before', 'arrival_1hr_before',
    'arrival_1hr_after', 'arrival_6hr_after'
]

# Ensure both dataframes are sorted by their respective 'datetime' columns
flights = flights.sort_values(by=['ORIGIN_AIRPORT'] + departure_columns + arrival_columns)
weather = weather.sort_values(by=['station', 'valid'])

# Define a function to apply merge_asof within each group (by 'code' or destination)
def merge_with_nearest_datetime(group_df1, group_df2, left_on, right_on):
    # Ensure both dataframes are sorted by the respective merge keys
    group_df1 = group_df1.sort_values(by=left_on)
    group_df2 = group_df2.sort_values(by=right_on)

    # Use merge_asof within the specific group
    return pd.merge_asof(group_df1, group_df2, left_on=left_on, right_on=right_on, direction='nearest')

# Initialize lists to store merged DataFrames for departures and arrivals
departure_merged_list = []
arrival_merged_list = []

# Apply merge_asof within each group for each departure time column
for departure_col in departure_columns:
    merged_departure = pd.concat([
        merge_with_nearest_datetime(group_df1, group_df2, left_on=departure_col, right_on='valid')
        for key, group_df1 in flights.groupby('ORIGIN_AIRPORT')
        for key2, group_df2 in weather.groupby('station')
        if key == key2  # Only merge matching airport codes
    ])

    # Reset index after merging
    merged_departure = merged_departure.reset_index(drop=True)

    # Add the prefix 'departure_' to all columns that came from the 'weather' DataFrame
    merged_departure.columns = [
        f"{departure_col}_{col}" if col not in flights.columns else col
        for col in merged_departure.columns
    ]

    # Remove 'valid' and 'station' columns from the departure merged dataframe, keeping only the columns with the prefix
    merged_departure = merged_departure.drop(
        columns=[col for col in merged_departure.columns if 'valid' in col or 'station' in col], errors='ignore')

    # Add to the list
    departure_merged_list.append(merged_departure)

# Apply merge_asof within each group for each arrival time column
for arrival_col in arrival_columns:
    merged_arrival = pd.concat([
        merge_with_nearest_datetime(group_df1, group_df2, left_on=arrival_col, right_on='valid')
        for key, group_df1 in flights.groupby('DESTINATION_AIRPORT')
        for key2, group_df2 in weather.groupby('station')
        if key == key2  # Only merge matching airport codes
    ])

    # Reset index after merging
    merged_arrival = merged_arrival.reset_index(drop=True)

    # Add the prefix 'arrival_' to all columns that came from the 'weather' DataFrame
    merged_arrival.columns = [
        f"{arrival_col}_{col}" if col not in flights.columns else col
        for col in merged_arrival.columns
    ]

    # Remove 'valid' and 'station' columns from the arrival merged dataframe, keeping only the columns with the prefix
    merged_arrival = merged_arrival.drop(
        columns=[col for col in merged_arrival.columns if 'valid' in col or 'station' in col], errors='ignore')

    # Add to the list
    arrival_merged_list.append(merged_arrival)

# Concatenate all merged departure and arrival data
departure_merged_df = pd.concat(departure_merged_list, axis=1)
arrival_merged_df = pd.concat(arrival_merged_list, axis=1)

# Extract new columns (departure weather data)
departure_weather_columns = [col for col in departure_merged_df.columns if col not in flights.columns]

# Extract new columns (arrival weather data)
arrival_weather_columns = [col for col in arrival_merged_df.columns if col not in flights.columns]

# Create final dataframe by concatenating flights with only new columns
merged_df = pd.concat([
    flights,
    departure_merged_df[departure_weather_columns],
    arrival_merged_df[arrival_weather_columns]
], axis=1)



In [32]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 460390 entries, 58 to 460385
Columns: 106 entries, YEAR to arrival_6hr_after_vsby
dtypes: datetime64[ns](12), float64(60), int64(11), object(23)
memory usage: 375.8+ MB


# **Missing Data, Dummies & Feature Engineering**

In [33]:
merged_df.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE_x', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE',
       ...
       'arrival_1hr_before_p01m', 'arrival_1hr_before_vsby',
       'arrival_1hr_after_tmpc', 'arrival_1hr_after_sped',
       'arrival_1hr_after_p01m', 'arrival_1hr_after_vsby',
       'arrival_6hr_after_tmpc', 'arrival_6hr_after_sped',
       'arrival_6hr_after_p01m', 'arrival_6hr_after_vsby'],
      dtype='object', length=106)

In [34]:
# List of columns to fill NaN values with 0
delay_columns = ['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
                 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']

# Fill NaN values in the specified columns with 0
merged_df[delay_columns] = merged_df[delay_columns].fillna(0)

In [35]:
# List of cancellation-related columns
cancellation_columns = ['CANCELLATION_REASON', 'CANCELLATION_DESCRIPTION']

# Fill NaN values in the specified columns with "Not cancelled"
merged_df[cancellation_columns] = merged_df[cancellation_columns].fillna("Not cancelled")

In [36]:
# Rename the 'AIRLINE_y' column to 'Airline'
merged_df.rename(columns={'AIRLINE_y': 'AIRLINE'}, inplace=True)

# Get dummies for the specified columns (0-indexed, so column 33 is 'AIRLINE_y', 35 is 'Origin_AIRPORT', 42 is 'Destination_AIRPORT', 48 is 'CANCELLATION_DESCRIPTION')
dummies = pd.get_dummies(merged_df[['AIRLINE', 'Origin_AIRPORT', 'Destination_AIRPORT', 'CANCELLATION_DESCRIPTION']], drop_first=True)

# Drop dummy columns
merged_df.drop(columns=merged_df[['AIRLINE', 'Origin_AIRPORT', 'Destination_AIRPORT', 'CANCELLATION_DESCRIPTION']], inplace=True)

# Concatenate the dummies with the original dataframe
merged_df = pd.concat([merged_df, dummies], axis=1)

In [37]:
# Define the columns you want to drop
columns_to_drop = ['YEAR', 'MONTH', 'DAY', 'AIRLINE_x', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'WHEELS_OFF',
       'WHEELS_ON', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
        'CANCELLATION_REASON',
        'date', 'IATA_CODE_x', 'IATA_CODE_y', 'Origin_CITY',
       'Origin_STATE', 'Origin_COUNTRY', 'Origin_LATITUDE', 'Origin_LONGITUDE',
       'Destination_IATA_CODE', 'Destination_CITY',
       'Destination_STATE', 'Destination_COUNTRY', 'Destination_LATITUDE',
       'Destination_LONGITUDE',
       'SCHEDULED_TIME_HOURS', 'ELAPSED_TIME_HOURS', 'Time Difference',
       'SCHEDULED_TIME_BOOL', 'ELAPSED_TIME_BOOL' ]

# Drop the specified columns
merged_df.drop(columns=columns_to_drop, inplace=True)


In [38]:
# Get the count of null values in each column
null_columns = merged_df.isnull().sum()

# Filter and display only columns with null values
null_columns_with_nulls = null_columns[null_columns > 0]

# Display columns with null values and the count of nulls
print(null_columns_with_nulls)

DEPARTURE_DELAY            6377
TAXI_OUT                   6640
SCHEDULED_TIME                1
ELAPSED_TIME               7964
AIR_TIME                   7964
TAXI_IN                    6862
ARRIVAL_DELAY              7964
DEPARTURE_TIME_DATETIME    6377
ARRIVAL_TIME_DATETIME      6862
dtype: int64


In [39]:
filtered_df = merged_df.loc[merged_df['CANCELLED'] == 1]

# Get the count of null values in each column
null_columns = filtered_df.isnull().sum()

# Filter and display only columns with null values
null_columns_with_nulls = null_columns[null_columns > 0]

# Display columns with null values and the count of nulls
print(null_columns_with_nulls)

DEPARTURE_DELAY            6377
TAXI_OUT                   6640
SCHEDULED_TIME                1
ELAPSED_TIME               6672
AIR_TIME                   6672
TAXI_IN                    6672
ARRIVAL_DELAY              6672
DEPARTURE_TIME_DATETIME    6377
ARRIVAL_TIME_DATETIME      6672
dtype: int64


In [40]:
filtered_df = merged_df.loc[merged_df['DIVERTED'] == 1]

# Get the count of null values in each column
null_columns = filtered_df.isnull().sum()

# Filter and display only columns with null values
null_columns_with_nulls = null_columns[null_columns > 0]

# Display columns with null values and the count of nulls
print(null_columns_with_nulls)

ELAPSED_TIME             1292
AIR_TIME                 1292
TAXI_IN                   190
ARRIVAL_DELAY            1292
ARRIVAL_TIME_DATETIME     190
dtype: int64


In [41]:
# First, handle cancelled flights
merged_df.loc[merged_df['CANCELLED'] == 1, ['DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'ELAPSED_TIME', 'AIR_TIME', 'TAXI_IN', 'TAXI_OUT']] = 0

# Handle diverted flights
merged_df.loc[merged_df['DIVERTED'] == 1, ['DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'ELAPSED_TIME', 'AIR_TIME', 'TAXI_IN', 'TAXI_OUT']] = 0

In [42]:
# Get the count of null values in each column
null_columns = merged_df.isnull().sum()

# Filter and display only columns with null values
null_columns_with_nulls = null_columns[null_columns > 0]

# Display columns with null values and the count of nulls
print(null_columns_with_nulls)

SCHEDULED_TIME                1
DEPARTURE_TIME_DATETIME    6377
ARRIVAL_TIME_DATETIME      6862
dtype: int64


In [43]:
# For cancelled flights, set datetime to NaT (Not a Time) or scheduled times
merged_df.loc[merged_df['CANCELLED'] == 1, 'DEPARTURE_TIME_DATETIME'] = merged_df['SCHEDULED_DEPARTURE_DATETIME']
merged_df.loc[merged_df['CANCELLED'] == 1, 'ARRIVAL_TIME_DATETIME'] = merged_df['SCHEDULED_ARRIVAL_DATETIME']

# For diverted flights, set datetime to scheduled times
merged_df.loc[merged_df['DIVERTED'] == 1, 'DEPARTURE_TIME_DATETIME'] = merged_df['SCHEDULED_DEPARTURE_DATETIME']
merged_df.loc[merged_df['DIVERTED'] == 1, 'ARRIVAL_TIME_DATETIME'] = merged_df['SCHEDULED_ARRIVAL_DATETIME']


In [44]:
# Get the count of null values in each column
null_columns = merged_df.isnull().sum()

# Filter and display only columns with null values
null_columns_with_nulls = null_columns[null_columns > 0]

# Display columns with null values and the count of nulls
print(null_columns_with_nulls)

SCHEDULED_TIME    1
dtype: int64


In [45]:
# Show rows where SCHEDULED_TIME is NaN
missing_value = merged_df[merged_df['SCHEDULED_TIME'].isna()]

In [46]:
# Show the row as a dictionary (for a single row with NaN in SCHEDULED_TIME)
row_dict = missing_value.iloc[0].to_dict()
print(row_dict)

{'DAY_OF_WEEK': 7, 'DEPARTURE_DELAY': 0.0, 'TAXI_OUT': 0.0, 'SCHEDULED_TIME': nan, 'ELAPSED_TIME': 0.0, 'AIR_TIME': 0.0, 'DISTANCE': 641, 'TAXI_IN': 0.0, 'ARRIVAL_DELAY': 0.0, 'DIVERTED': 0, 'CANCELLED': 1, 'AIR_SYSTEM_DELAY': 0.0, 'SECURITY_DELAY': 0.0, 'AIRLINE_DELAY': 0.0, 'LATE_AIRCRAFT_DELAY': 0.0, 'WEATHER_DELAY': 0.0, 'SCHEDULED_DEPARTURE_DATETIME': Timestamp('2015-04-26 20:59:00'), 'DEPARTURE_TIME_DATETIME': Timestamp('2015-04-26 20:59:00'), 'ARRIVAL_TIME_DATETIME': Timestamp('2015-04-26 23:50:00'), 'SCHEDULED_ARRIVAL_DATETIME': Timestamp('2015-04-26 23:50:00'), 'departure_6hr_before': Timestamp('2015-04-26 14:59:00'), 'departure_1hr_before': Timestamp('2015-04-26 19:59:00'), 'departure_1hr_after': Timestamp('2015-04-26 21:59:00'), 'departure_6hr_after': Timestamp('2015-04-27 02:59:00'), 'arrival_6hr_before': Timestamp('2015-04-26 17:50:00'), 'arrival_1hr_before': Timestamp('2015-04-26 22:50:00'), 'arrival_1hr_after': Timestamp('2015-04-27 00:50:00'), 'arrival_6hr_after': Times

In [47]:
# Set the missing value of 'SCHEDULED_TIME' to 111 for the row where it is NaN
# The missing value is calculated with the other data avaiable
merged_df.loc[merged_df['SCHEDULED_TIME'].isna(), 'SCHEDULED_TIME'] = 111

In [48]:
# Get the count of null values in each column
null_columns = merged_df.isnull().sum()

# Filter and display only columns with null values
null_columns_with_nulls = null_columns[null_columns > 0]

# Display columns with null values and the count of nulls
print(null_columns_with_nulls)

Series([], dtype: int64)


In [ ]:
# Create the WEEKEND column
merged_df['WEEKEND'] = merged_df['DAY_OF_WEEK'].apply(lambda x: 1 if x in [6, 7] else 0)

# Drop the DAY_OF_THE_WEEK column
merged_df = merged_df.drop(columns=['DAY_OF_WEEK'])

In [51]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 460390 entries, 58 to 460385
Data columns (total 100 columns):
 #   Column                                                                Non-Null Count   Dtype         
---  ------                                                                --------------   -----         
 0   DEPARTURE_DELAY                                                       460390 non-null  float64       
 1   TAXI_OUT                                                              460390 non-null  float64       
 2   SCHEDULED_TIME                                                        460390 non-null  float64       
 3   ELAPSED_TIME                                                          460390 non-null  float64       
 4   AIR_TIME                                                              460390 non-null  float64       
 5   DISTANCE                                                              460390 non-null  int64         
 6   TAXI_IN                        

In [52]:
merged_df.to_csv('/content/drive/MyDrive/HSG/project/final_df.csv', index=False)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/raphael/flights_final.csv')

In [3]:
average_weather_delay = pd.DataFrame(df.groupby('FLIGHT_NUMBER')['WEATHER_DELAY'].mean().dropna())

In [4]:
average_weather_delay['FLIGHT_NUMBER'] = average_weather_delay.index

In [5]:
average_weather_delay

,WEATHER_DELAY,FLIGHT_NUMBER
FLIGHT_NUMBER,,
3,10.615385,3
5,0.000000,5
6,0.000000,6
7,6.175000,7
12,1.243590,12
...,...,...
6790,0.000000,6790
6829,0.000000,6829
6839,0.000000,6839


In [6]:
# Group by 'FLIGHT_NUMBER' and create the dictionary
flight_dict = {
    flight_number: list(
        group[['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'AIRLINE_x']].drop_duplicates().itertuples(index=False, name=None)
    )
    for flight_number, group in df.groupby('FLIGHT_NUMBER')
}

# Display the resulting dictionary
flight_dict

{1: [('LAX', 'PHX', 'WN')],
 2: [('SFO', 'PHX', 'WN')],
 3: [('LAX', 'ORD', 'UA'),
  ('LAS', 'ORD', 'UA'),
  ('LGA', 'DEN', 'UA'),
  ('ORD', 'LAS', 'UA'),
  ('ORD', 'SFO', 'UA'),
  ('DEN', 'SFO', 'UA'),
  ('ATL', 'ORD', 'UA')],
 4: [('DEN', 'PHX', 'WN')],
 5: [('PHX', 'LAX', 'WN')],
 6: [('PHX', 'SFO', 'WN')],
 7: [('ATL', 'LGA', 'WN'),
  ('LGA', 'DEN', 'WN'),
  ('ATL', 'DEN', 'WN'),
  ('DEN', 'SFO', 'WN'),
  ('SFO', 'LAS', 'WN')],
 11: [('ATL', 'DEN', 'WN'), ('SFO', 'LAX', 'WN'), ('LAX', 'LAS', 'WN')],
 12: [('SFO', 'PHX', 'UA'),
  ('ORD', 'LAX', 'UA'),
  ('DEN', 'ORD', 'UA'),
  ('DEN', 'LAX', 'UA'),
  ('LGA', 'DEN', 'WN'),
  ('ATL', 'DEN', 'UA')],
 14: [('PHX', 'LAX', 'WN')],
 15: [('PHX', 'SFO', 'WN')],
 16: [('LAX', 'ATL', 'DL')],
 17: [('ATL', 'LAX', 'DL')],
 18: [('PHX', 'SFO', 'WN'), ('LAX', 'SFO', 'WN')],
 21: [('PHX', 'DEN', 'WN')],
 22: [('SFO', 'LAS', 'WN')],
 23: [('DFW', 'SFO', 'AA'),
  ('SFO', 'DFW', 'AA'),
  ('LAX', 'LAS', 'AA'),
  ('LAS', 'LAX', 'AA'),
  ('SFO', 'DEN', 

In [7]:
# Extract flight numbers present in DataFrame A
flight_numbers_in_average = set(average_weather_delay['FLIGHT_NUMBER'])

# Filter Dictionary B
filtered_flight_dict = {
    flight_num: routes
    for flight_num, routes in flight_dict.items()
    if flight_num in flight_numbers_in_average
}

# Create delay mapping
delay_mapping = pd.Series(average_weather_delay.WEATHER_DELAY.values, index=average_weather_delay.FLIGHT_NUMBER).to_dict()

# Augment the filtered dictionary with average weather delay
augmented_flight_dict = {
    flight_num: {
        "routes": routes,
        "AVERAGE_WEATHER_DELAY": delay_mapping.get(flight_num, None)
    }
    for flight_num, routes in filtered_flight_dict.items()
}

augmented_flight_dict

{3: {'routes': [('LAX', 'ORD', 'UA'),
   ('LAS', 'ORD', 'UA'),
   ('LGA', 'DEN', 'UA'),
   ('ORD', 'LAS', 'UA'),
   ('ORD', 'SFO', 'UA'),
   ('DEN', 'SFO', 'UA'),
   ('ATL', 'ORD', 'UA')],
  'AVERAGE_WEATHER_DELAY': 10.615384615384615},
 5: {'routes': [('PHX', 'LAX', 'WN')], 'AVERAGE_WEATHER_DELAY': 0.0},
 6: {'routes': [('PHX', 'SFO', 'WN')], 'AVERAGE_WEATHER_DELAY': 0.0},
 7: {'routes': [('ATL', 'LGA', 'WN'),
   ('LGA', 'DEN', 'WN'),
   ('ATL', 'DEN', 'WN'),
   ('DEN', 'SFO', 'WN'),
   ('SFO', 'LAS', 'WN')],
  'AVERAGE_WEATHER_DELAY': 6.175},
 12: {'routes': [('SFO', 'PHX', 'UA'),
   ('ORD', 'LAX', 'UA'),
   ('DEN', 'ORD', 'UA'),
   ('DEN', 'LAX', 'UA'),
   ('LGA', 'DEN', 'WN'),
   ('ATL', 'DEN', 'UA')],
  'AVERAGE_WEATHER_DELAY': 1.2435897435897436},
 16: {'routes': [('LAX', 'ATL', 'DL')], 'AVERAGE_WEATHER_DELAY': 0.0},
 17: {'routes': [('ATL', 'LAX', 'DL')], 'AVERAGE_WEATHER_DELAY': 9.359375},
 21: {'routes': [('PHX', 'DEN', 'WN')], 'AVERAGE_WEATHER_DELAY': 0.0},
 23: {'routes': [(

In [8]:
augmented_flight_dict.items()

dict_items([(3, {'routes': [('LAX', 'ORD', 'UA'), ('LAS', 'ORD', 'UA'), ('LGA', 'DEN', 'UA'), ('ORD', 'LAS', 'UA'), ('ORD', 'SFO', 'UA'), ('DEN', 'SFO', 'UA'), ('ATL', 'ORD', 'UA')], 'AVERAGE_WEATHER_DELAY': 10.615384615384615}), (5, {'routes': [('PHX', 'LAX', 'WN')], 'AVERAGE_WEATHER_DELAY': 0.0}), (6, {'routes': [('PHX', 'SFO', 'WN')], 'AVERAGE_WEATHER_DELAY': 0.0}), (7, {'routes': [('ATL', 'LGA', 'WN'), ('LGA', 'DEN', 'WN'), ('ATL', 'DEN', 'WN'), ('DEN', 'SFO', 'WN'), ('SFO', 'LAS', 'WN')], 'AVERAGE_WEATHER_DELAY': 6.175}), (12, {'routes': [('SFO', 'PHX', 'UA'), ('ORD', 'LAX', 'UA'), ('DEN', 'ORD', 'UA'), ('DEN', 'LAX', 'UA'), ('LGA', 'DEN', 'WN'), ('ATL', 'DEN', 'UA')], 'AVERAGE_WEATHER_DELAY': 1.2435897435897436}), (16, {'routes': [('LAX', 'ATL', 'DL')], 'AVERAGE_WEATHER_DELAY': 0.0}), (17, {'routes': [('ATL', 'LAX', 'DL')], 'AVERAGE_WEATHER_DELAY': 9.359375}), (21, {'routes': [('PHX', 'DEN', 'WN')], 'AVERAGE_WEATHER_DELAY': 0.0}), (23, {'routes': [('DFW', 'SFO', 'AA'), ('SFO', 'D

In [30]:
import pickle

# Save the dictionary to a pickle file
with open("../data/inference/flight_numbers.pkl", "wb") as file:
    pickle.dump(augmented_flight_dict, file)